<a href="https://colab.research.google.com/github/stirlitzzz/graph_test/blob/main/gpt_uvm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###UVM model

In [31]:
import numpy as np

def generate_recombinant_tree(S, T, r, sigma, num_time_steps=20):
    dt = T / num_time_steps


    # Initialize the 2D grid for the recombinant tree
    grid = np.zeros((2 * num_time_steps + 1, num_time_steps + 1))

    # Set the first column of the grid based on the provided formula
    x_values = np.arange(-num_time_steps, num_time_steps + 1)
    grid[:, 0] = S * np.exp(x_values * sigma * np.sqrt(dt))

    # Generate the rest of the recombinant tree using broadcasting
    r_exp = np.exp(r * dt)
    for t in range(1, num_time_steps + 1):
        grid[:, t] = grid[:, t - 1] * r_exp

    return grid.T

def trinomial_transitions_av(dt,sigma,sigma_max):
    u=np.exp(sigma_max * np.sqrt(dt)+r*dt)
    d=np.exp(-sigma_max * np.sqrt(dt)+r*dt)
    m=np.exp(r*dt)
    return u, d,m

def trinomial_transition_probabilities_av(dt, r, sigma, sigma_max):
    p = (sigma ** 2) / (2 * sigma_max ** 2)
    pu = p*(1-sigma_max * np.sqrt(dt)/2)
    pd = p*(1+sigma_max * np.sqrt(dt)/2) 

    pm = 1 - 2*p
    return pu, pm, pd

def initialize_option_grid(spot_grid, strike, T, r, cp):
    num_time_steps, num_spots = spot_grid.shape
    dt = T / (num_time_steps - 1)
    option_grid = np.zeros((num_time_steps, num_spots))

    # Set boundary conditions for the last time step
    if cp == 'c':
        option_grid[-1, :] = np.maximum(spot_grid[-1, :] - strike, 0)
    elif cp == 'p':
        option_grid[-1, :] = np.maximum(strike - spot_grid[-1, :], 0)
    else:
        raise ValueError("cp must be either 'c' for call or 'p' for put.")

    # Set boundary conditions for all time steps for the highest and lowest spot
    for t in range(num_time_steps):
        if cp == 'c':
            option_grid[t, 0] = np.maximum(spot_grid[t, 0] - strike, 0) * np.exp(-r * (T - t * dt))
            option_grid[t, -1] = np.maximum(spot_grid[t, -1] - strike, 0) * np.exp(-r * (T - t * dt))
        elif cp == 'p':
            option_grid[t, 0] = np.maximum(strike - spot_grid[t, 0], 0) * np.exp(-r * (T - t * dt))
            option_grid[t, -1] = np.maximum(strike - spot_grid[t, -1], 0) * np.exp(-r * (T - t * dt))

    return option_grid


def create_L_operator(dt, sigma):
    w_up = (1 - sigma * np.sqrt(dt) / 2)
    w_dn = (1 + sigma * np.sqrt(dt) / 2)
    w_md = -1
    #the division by two is not 

    return w_dn, w_md, w_up







In [32]:
def uncertain_volatility_pde_portfolio_v2(S,cp, strikes, qtys, T, r, vol_min, vol_max, num_grid_points=200):
    """
    Estimate the prices, deltas, and gammas of European options in a portfolio using the uncertain volatility model
    and an explicit finite difference method.

    Parameters:
    S (float): Current stock price
    cp (list of str): Call or put ('c' or 'p')
    strikes (list of float): List of option strike prices
    qtys (list of int): List of option quantities
    T (float): Time to expiration (in years)
    r (float): Annual risk-free interest rate
    vol_min (float): Minimum annual stock price volatility (as a decimal)
    vol_max (float): Maximum annual stock price volatility (as a decimal)
    num_grid_points (int): Number of grid points for the stock price

    Returns:
    tuple: Tuple containing the following lists
    (prices, deltas, gammas): Prices, deltas, and gammas of the European options in the portfolio
    """
    spot_grid = generate_recombinant_tree(num_time_steps=num_grid_points, S=S, sigma=vol_max, T=T, r=r)
    option_grid = initialize_option_grid(spot_grid, strikes[0], T, r, cp[0])

    option_grids=np.stack([initialize_option_grid(spot_grid, strikes[i], T, r, cp[i]) for i in range(len(cp))])
    portfolio_grid=np.zeros(option_grids[0,:,:].shape)

    portfolio_grid[-1,:]=np.einsum('ij,i->j',option_grids[:,-1,:],qtys)
    dt = T / (num_grid_points)

    pu, pm, pd = trinomial_transition_probabilities_av(dt,r, vol_max, vol_max)

    pu_min=pu*vol_min**2/(vol_max**2)
    pd_min=pd*vol_min**2/(vol_max**2)
    pm_min=1-pu_min-pd_min
    
    w_max=np.array([pu,pm,pd])
    w_min=np.array([pu_min,pm_min,pd_min])

    portfolio_gamma=np.zeros(portfolio_grid.shape[1])
    L_operator=np.array(create_L_operator(dt,vol_max))

    for t in range(num_grid_points - 1, -1, -1):
        #1 calculate the gamma operator for the t+1 step
        #2 chose the right transition probabilities for each element based on vol
        #3 add up the weighted probabilities for each option
        
        portfolio_gamma=np.convolve(portfolio_grid[t+1,:],L_operator,mode='same')
        for opt in range(len(cp)):
            prev_values=option_grids[opt,t+1,:]
            new_max_vol_values=np.convolve(prev_values,w_max,mode='same')
            new_min_vol_values=np.convolve(prev_values,w_min,mode='same')
            option_grids[opt,t,1:-2]=np.where(portfolio_gamma[1:-2]<0,new_max_vol_values[1:-2],new_min_vol_values[1:-2])*np.exp(-r*dt)
        portfolio_grid[t,:]=np.einsum('ij,i->j',option_grids[:,t,:],qtys)
    
    option_prices=np.zeros(len(cp))
    option_deltas=np.zeros(len(cp))
    option_gammas=np.zeros(len(cp))

    for (ind, qty) in enumerate(qtys):
        K=strikes[ind]
        option_grid = option_grids[ind,:,:]
        price=np.interp(S, spot_grid[0, :], option_grid[0, :])
        delta=np.interp(S, spot_grid[0,1:-2], (option_grid[0,2:-1]-option_grid[0,0:-3])/(spot_grid[0,2:-1]-spot_grid[0,0:-3]))
        option_prices[ind]=price
        option_deltas[ind]=delta
    return (spot_grid, option_grids, option_prices, option_deltas)

###Inputs

In [36]:
S= 100
cp=['p','c']
strikes = [90, 110]
qtys = [-1, 1]
T = 1 
r = 0.05
vol_min = 0.2
vol_max = 0.2



###Price

In [41]:
import py_vollib_vectorized as pv

(spot_grid,option_grid,option_prices,option_deltas) = uncertain_volatility_pde_portfolio_v2(S,cp, strikes, qtys, T, r, vol_min, vol_max)
implied_vol = pv.implied_volatility.vectorized_implied_volatility(option_prices, S, strikes, T, r, flag=cp,q=0,model='black_scholes_merton')
all_greeks=pv.get_all_greeks(cp, S, strikes, T, r, implied_vol.IV, model='black_scholes_merton',q=0, return_as='dataframe')




In [42]:
#create output dataframe with the following columns:
#qty, spot price, strike price, call_put, option price, uvm delta, uvm gamma, bs implied vol, bs delta, bs gamma
import pandas as pd

df=pd.DataFrame(columns=['qty','spot','T','strike','call_put','bs_implied_vol','option_price','option_price_pct','uvm_delta','bs_delta','uvm_gamma','bs_gamma'])
for i in range(len(cp)):
    df.loc[i,'qty']=qtys[i]
    df.loc[i,'spot']=S
    df.loc[i,'T']=T
    df.loc[i,'strike']=strikes[i]
    df.loc[i,'call_put']=cp[i]
    df.loc[i,'option_price']=option_prices[i]
    df.loc[i,'option_price_pct']=option_prices[i]/S
    df.loc[i,'uvm_delta']=option_deltas[i]
    df.loc[i,'bs_implied_vol']=implied_vol.IV[i]

    df.loc[i,'bs_delta']=all_greeks.loc[i,'delta']
    df.loc[i,'bs_gamma']=all_greeks.loc[i,'gamma']*((S/(100))) #the scaling might be wrong

formatted_df = df[['qty','spot','T','strike','call_put','bs_implied_vol','option_price','option_price_pct','uvm_delta','bs_delta']].style.format({
    'qty': '{:,.0f}',
    'spot': '{:.2f}',
    'T': '{:.2f}',
    'strike': '{:.2f}',
    'call_put': '{:^10}',
    'option_price': '{:.2f}',
    'option_price_pct': '{:.2%}',
    'uvm_delta': '{:.2%}',
    'uvm_gamma': '{:.2%}',
    'bs_implied_vol': '{:.2%}',
    'bs_delta': '{:.2%}',
    'bs_gamma': '{:.2%}'

})


In [43]:
display(formatted_df)

,qty,spot,T,strike,call_put,bs_implied_vol,option_price,option_price_pct,uvm_delta,bs_delta
0,-1,100.00,1.00,90.00,p,20.02%,2.32,2.32%,-19.03%,-19.05%
1,1,100.00,1.00,110.00,c,20.02%,6.05,6.05%,44.99%,44.98%
